##### Imports

In [2]:
from __future__ import division
import math
import pyaudio
import numpy as np
from midiutil import MIDIFile
import copy
import operator
from collections import deque

##### Methods

In [3]:
def play_note(INPUT_NOTE, INPUT_TIME):
    SAMPLES_PER_SECOND = 44100
    NOTE_TIME_SECONDS = INPUT_TIME
    
    CYCLES_PER_SECOND = SAMPLES_PER_SECOND / INPUT_NOTE
    NUM_SAMPLES = SAMPLES_PER_SECOND * NOTE_TIME_SECONDS
    audio = pyaudio.PyAudio()
    
    stream = audio.open(
        format=pyaudio.paFloat32,
        channels=1,
        rate=SAMPLES_PER_SECOND,
        output=True,
    )
    
    samples = (np.sin(2*np.pi*np.arange(SAMPLES_PER_SECOND*INPUT_TIME)*INPUT_NOTE/SAMPLES_PER_SECOND)).astype(np.float32)
    
    stream = audio.open(
        format=pyaudio.paFloat32,
        channels=1,
        rate=SAMPLES_PER_SECOND,
        output=True,
    )

    byte_array = bytearray()
    
    chunks = samples
    fade = 2000

    fade_in = np.arange(0., 1., 1/fade)
    fade_out = np.arange(1., 0., -1/fade)
    
    samples[:fade] = np.multiply(samples[:fade], fade_in)
    samples[-fade:] = np.multiply(samples[-fade:], fade_out)

    stream.write(samples.astype(np.float32).tobytes())
    #stream.close()
    #audio.terminate()
    
#reads a configuration file for note names and frequencies
def read_config(file):
    #importing note config file
    with open(file, "r") as file:
        config = file.readlines()
        file.close()
    
    #stripping \n tags
    data = [x.strip() for x in config]
    return data

#creates the dictionary of note-frequency pairs
def create_dict(config):
    notes_temp = []
    note_pairs = {}

    #splits into [note,frequency]
    for i in range(0,len(config)):
        notes_temp.append(config[i].split(":"))

    #creates dictionaries and adds to list
    for j in range(0,len(config)):
        for k in range(0,2):
            #storing key and value for this index
            key = notes_temp[j][0]
            value = (notes_temp[j][1])
        #hopefully adding the dictionary element
        note_pairs[key] = value
        
    return note_pairs

#converts note name to frequency using the dictionary
def convert(note_name, book):
    return float(book.get(note_name))

def random_key_sig(full_map):
    major_steps = [2,2,1,2,2,2,1]
    minor_steps = [2,1,2,2,1,2,2]
    
    #choosing a random scale to use
    scales = [major_steps,minor_steps]
    seed = np.random.randint(0,11)
    
    if(np.random.choice([0,1]) == 0):
        choice = major_steps
    else:
        choice = minor_steps
    
    key_notes = []
    idx = seed
    ex_count = 0
    key_notes.append(idx)
    
    #generating indicies of appropriate notes in key signature
    while(max(key_notes)<len(full_map)-2):
        if(ex_count == len(choice)):
            ex_count = 0
            
        idx+=choice[ex_count]
        ex_count+=1
        
        key_notes.append(idx)
    
    new_list = []
    full_copy = copy.deepcopy(full_map)
    
    #generating list of only notes in key_signature
    keys_list = list(full_copy.keys())
    for i in range(0,len(key_notes)):
        new_list.append(keys_list[int(key_notes[i])])
    #print(new_list)
    
    new_dict = {}
    new_freq = []
    
    #getting associated frequencies to key signature notes
    for j in new_list:
        new_freq.append(full_map.get(j))
        
    #putting together the dictionary
    for k, val in enumerate(new_list,0):
        key = val
        value = new_freq[k]
        new_dict[key] = value
        
    return new_dict

In [5]:
note_data = read_config("note_config.txt");
pairs = create_dict(note_data);

print(pairs)

{'C1': '32.70', 'C#1': '34.65', 'D1': '36.71', 'D#1': '38.89', 'E1': '41.20', 'F1': '43.65', 'F#1': '46.25', 'G1': '49.00', 'G#1': '51.91', 'A1': '55.00', 'A#1': '58.27', 'B1': '61.74', 'C2': '65.41', 'C#2': '69.30', 'D2': '73.42', 'D#2': '77.78', 'E2': '82.41', 'F2': '87.31', 'F#2': '92.50', 'G2': '98.00', 'G#2': '103.83', 'A2': '110.00', 'A#2': '116.54', 'B2': '123.47', 'C3': '130.81', 'C#3': '138.59', 'D3': '146.83', 'D#3': '155.56', 'E3': '164.81', 'F3': '174.61', 'F#3': '185.00', 'G3': '196.00', 'G#3': '207.65', 'A3': '220.00', 'A#3': '233.08', 'B3': '246.94', 'C4': '261.63', 'C#4': '277.18', 'D4': '293.66', 'D#4': '311.13', 'E4': '329.63', 'F4': '349.23', 'F#4': '369.99', 'G4': '392.00', 'G#4': '415.30', 'A4': '440.00', 'A#4': '466.16', 'B4': '493.88', 'C5': '523.25', 'C#5': '554.37', 'D5': '587.33', 'D#5': '622.25', 'E5': '659.25', 'F5': '698.46', 'F#5': '739.99', 'G5': '783.99', 'G#5': '830.61', 'A5': '880.00', 'A#5': '932.33', 'B5': '987.77', 'C6': '1046.50', 'C#6': '1108.73',

In [6]:
file = "scp761-dh.txt"
raw_text_c = open(file, 'r', encoding='utf-8').read()

clean_text_c = raw_text_c.replace('\xa0',' ')
clean_text_c = clean_text_c.lower()

reject_chars = ['€','é','î','³','•','▸','◂','|','°','º','»',
                '«','…','>','<','—','‘','’','“','”','–','~',
                '^', '$', '+','█','-',"'",',','.','/','?',';',':',
               '1','2','3','4','5','6','7','8','9','0','#']

for i in reject_chars:
    clean_text_c = clean_text_c.replace(i, '')
    
chars = sorted(list(set(clean_text_c)))

print(chars)
print(len(chars))

['\n', ' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
28


In [7]:
def random_char_map(char_set, note_dict):
    stack = []
    new_pairs = {}
    
    #creates stack of useable frequencies
    for i in note_dict:
        stack.append(note_dict[i])
          
    #creates dictionary of char-note pairs
    for j in char_set:
        key = j
        value = stack.pop(np.random.randint(0,len(stack)))
        new_pairs[key] = value
        
    return new_pairs

In [8]:
mappo = random_char_map(chars,pairs)
print(mappo)

mappo2 = random_char_map(chars,random_key_sig(pairs))

print('\n', mappo2)

{'\n': '110.00', ' ': '58.27', 'a': '1046.50', 'b': '1244.51', 'c': '77.78', 'd': '98.00', 'e': '1479.98', 'f': '3322.44', 'g': '2793.83', 'h': '1108.73', 'i': '830.61', 'j': '87.31', 'k': '123.47', 'l': '103.83', 'm': '38.89', 'n': '69.30', 'o': '261.63', 'p': '1318.51', 'q': '1661.22', 'r': '32.70', 's': '987.77', 't': '2637.02', 'u': '1760.00', 'v': '293.66', 'w': '43.65', 'x': '932.33', 'y': '220.00', 'z': '82.41'}

 {'\n': '65.41', ' ': '43.65', 'a': '87.31', 'b': '554.37', 'c': '1396.91', 'd': '783.99', 'e': '174.61', 'f': '196.00', 'g': '207.65', 'h': '103.83', 'i': '1864.66', 'j': '1046.50', 'k': '130.81', 'l': '277.18', 'm': '349.23', 'n': '622.25', 'o': '523.25', 'p': '392.00', 'q': '3322.44', 'r': '2489.02', 's': '2093.00', 't': '1661.22', 'u': '77.78', 'v': '2793.83', 'w': '3135.96', 'x': '466.16', 'y': '233.08', 'z': '116.54'}


In [11]:
def freq_to_midnum(freq):
    note = 12*np.log2(freq/(440/32))+9
    return int(round(note))
    
def text_to_music(input_str,final_map, t_notes):
    char_arr = list(input_str)
    lengths = [0.25,0.5,0.75,1.0]
    
    midnum_arr = []
    length_arr = []
    
    for i in range(0,t_notes):
        length = np.random.choice(lengths)
        #play_note(convert(char_arr[i],final_map),length)
        midnum_arr.append(freq_to_midnum(convert(char_arr[i],final_map)))
        length_arr.append(length)
        
    return midnum_arr, length_arr
        
def music_to_midi(degree_arr,dur_arr,file):
    
    track = 0
    channel = 1
    time = 0
    tempo = np.random.randint(100,160)
    volume = 100
    
    MyMIDI = MIDIFile(1)
    MyMIDI.addTempo(track,time,tempo)
    
    for i, pitch in enumerate(degree_arr):
        MyMIDI.addNote(track,channel,pitch,time+i,1,volume)
        
    with open(file, "wb") as output_file:
        MyMIDI.writeFile(output_file)

In [19]:
def text_to_midi(inputfile,config,savefile,complength):
    
    #read files, create map
    data = read_config(config)
    main_dict = create_dict(data)
    
    #open text file to convert
    raw_text_c = open(inputfile, 'r', encoding='utf-8').read()
    clean_text_c = raw_text_c.replace('\xa0',' ')
    clean_text_c = clean_text_c.lower()
    
    #clean file
    reject_chars = ['€','é','î','³','•','▸','◂','|','°','º','»',
                    '«','…','>','<','—','‘','’','“','”','–','~',
                    '^', '$', '+','█','-',"'",',','.','/','?',';',':',
                   '1','2','3','4','5','6','7','8','9','0','#','@','&','*']
    for i in reject_chars:
        clean_text_c = clean_text_c.replace(i, '')
    
    #remaining characters
    chars = sorted(list(set(clean_text_c)))
    
    #create charmap using set key signature
    charmap = random_char_map(chars, random_key_sig(main_dict))
    
    a1, a2 = text_to_music(clean_text_c,charmap, complength)
    music_to_midi(a1,a2,savefile)

In [15]:
for i in range(0,10):
    arr_1, arr_2 = text_to_music(clean_text_c,mappo2, 50)
    music_to_midi(arr_1, arr_2, "t-"+str(i)+".mid")
    print("Saving song " + str(i) + "...")

Saving song 0...
Saving song 1...
Saving song 2...
Saving song 3...
Saving song 4...
Saving song 5...
Saving song 6...
Saving song 7...
Saving song 8...
Saving song 9...


In [24]:
text_to_midi("Input/bee.txt","note_config.txt","Output/bee.mid",50)